In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

In [3]:
from xml.dom.minidom import parseString

In [4]:
import bz2

In [5]:
with bz2.open("data/jawikinews-20170201-pages-articles.xml.bz2") as fp:
    dom = parseString(fp.read())

In [6]:
import re

In [7]:
ascii = re.compile(r"^[!-~]+$")
def check_text(title, text):
    if len(title.childNodes) == 0 or len(text.childNodes) == 0:
        return False
    
    title = title.childNodes[0].data
    text = text.childNodes[0].data
    
    if ascii.match(title):
        # タイトルに全角文字を含まないものは除外
        return False
    if title.startswith("MediaWiki:"):
        return False
    if title.startswith("モジュール:"):
        return False
    if title.startswith("短信:"):
        return False
    if title.startswith("ファイル:"):
        return False
    if title.startswith("テンプレート:"):
        return False
    if title.startswith("ウィキニュース:"):
        return False
    if title.startswith("カテゴリ:"):
        return False
    if title.startswith("ポータル:"):
        return False
    if title.startswith("ヘルプ:"):
        return False
    if text.lower().startswith("#redirect") or text.startswith("#転送"):
        return False
    if title == "メインページ" or title == "オールターナティブメインページ":
        return False
    return True

titles = []
texts = []
for page in dom.getElementsByTagName("page"):
    title = page.getElementsByTagName("title")[0]
    text = page.getElementsByTagName("text")[0]
    
    if check_text(title, text):
        titles.append(title.childNodes[0].data)
        texts.append(text.childNodes[0].data)

In [8]:
! pip install mwparserfromhell

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
import mwparserfromhell as mw

In [10]:
wp_link = re.compile(r"{{W\|(.*?)}}")
wp_image = re.compile(r"\[\[[^\n]*?(thumb|thumbnail|frame)\|.*?\]\]", re.MULTILINE | re.DOTALL)
wp_category = re.compile(r"\[\[Category:.*?\]\]")
wp_magic = re.compile(r"__(.*?)__")
def normalize(text):
    return wp_magic.sub(r"", wp_category.sub(r"", wp_image.sub(r"", wp_link.sub(r"\1", text))))

In [11]:
contents = []
for i, text in enumerate(texts):
    content = mw.parse(normalize(text))
    content = content.strip_code()
    contents.append(content)

In [12]:
print(texts[0])

{{日付|2005年7月15日}}

[[画像:Location ShiretokoPeninJp.jpg|thumb|今回世界遺産登録された7物件のひとつ、知床半島の位置]]

2005年7月14日、[[w:南アフリカ共和国|南アフリカ共和国]][[w:ダーバン|ダーバン]]で開催中の第29回世界遺産会議で、[[w:ノルウェー|ノルウェー]]の[[w:フィヨルド|フィヨルド]]と[[w:日本|日本]]の[[w:知床半島|知床半島]]（[[w:北海道|北海道]]）含め世界7カ所が[[w:ユネスコ|ユネスコ]]の[[w:世界遺産|世界自然遺産]]に登録された。また既存2カ所の自然遺産の登録範囲の拡張が認められた。この追加により、登録された世界遺産の数は、自然遺産160、複合遺産24となる。

知床半島は、[[w:日本|日本]]国内では、[[w:屋久島|屋久島]]、[[w:白神山地|白神山地]]につぐ三番目の世界自然遺産になった。北海道内では、文化遺産、複合遺産をあわせ、はじめての世界遺産登録である。<!--登録された範囲は……（具体的な数字と行政地名）。-->

知床半島は[[w:羅臼岳|羅臼岳]](1660m)などを含む知床火山層を中心に形成され、[[w:オホーツク海|オホーツク海]]に突き出た半島である。山間部には針葉樹のエゾマツ、トドマツや広葉樹のダケカンバ、ミズナラの原生林がしげり、山頂部や稜線には高山植物群落がみられる。また冬季には流氷がみられ、海面が凍結する。知床半島の世界遺産登録に際しては、こうした自然環境における特色ある豊かな[[w:生態系|生態系]]、とりわけ[[w:絶滅危惧種|絶滅危惧種]]を含む生物種の保存への意義が高く評価された。知床半島に生息する動物には、トドや[[w:クジラ目|クジラ]]などの海域に生息する哺乳類、[[w:ヒグマ|ヒグマ]]、[[w:シマフクロウ|シマフクロウ]]を含む野鳥類などがある。

知床半島中部から先の山岳部と海岸部は1964年知床国立公園に指定された。その後、知床岬周辺は規制の厳しい特別保護地区に指定されるとともに立ち入りが禁止されている。（日本語版ウィキペディア「知床半島」より）。

今回登録された世界自然遺産は、先の2つを含め、以下の7つである。
*フレーデフォート・ドーム（[[w:南アフリカ|南アフリカ]]）

In [13]:
print(contents[0])

2005年7月14日、南アフリカ共和国ダーバンで開催中の第29回世界遺産会議で、ノルウェーのフィヨルドと日本の知床半島（北海道）含め世界7カ所がユネスコの世界自然遺産に登録された。また既存2カ所の自然遺産の登録範囲の拡張が認められた。この追加により、登録された世界遺産の数は、自然遺産160、複合遺産24となる。

知床半島は、日本国内では、屋久島、白神山地につぐ三番目の世界自然遺産になった。北海道内では、文化遺産、複合遺産をあわせ、はじめての世界遺産登録である。

知床半島は羅臼岳(1660m)などを含む知床火山層を中心に形成され、オホーツク海に突き出た半島である。山間部には針葉樹のエゾマツ、トドマツや広葉樹のダケカンバ、ミズナラの原生林がしげり、山頂部や稜線には高山植物群落がみられる。また冬季には流氷がみられ、海面が凍結する。知床半島の世界遺産登録に際しては、こうした自然環境における特色ある豊かな生態系、とりわけ絶滅危惧種を含む生物種の保存への意義が高く評価された。知床半島に生息する動物には、トドやクジラなどの海域に生息する哺乳類、ヒグマ、シマフクロウを含む野鳥類などがある。

知床半島中部から先の山岳部と海岸部は1964年知床国立公園に指定された。その後、知床岬周辺は規制の厳しい特別保護地区に指定されるとともに立ち入りが禁止されている。（日本語版ウィキペディア「知床半島」より）。

今回登録された世界自然遺産は、先の2つを含め、以下の7つである。
フレーデフォート・ドーム（南アフリカ）
ワジ・アル＝ヒタンの鯨渓谷（エジプト）
知床（日本）
西ノルウェーのフィヨルド、ガイランゲル・フィヨルドおよびナールオイ・フィヨルド（ノルウェー）
カリフォルニア湾の諸島および保護地域（メキシコ）
ドンパヤーイェン――カオヤイ森林地帯（タイ）
コイバ国立公園とその海洋保護特別地域（パナマ）
また、以下の2つの世界自然遺産の登録範囲が拡張された。
フラワーズ渓谷国立公園（インド）
セント・キルダ（イギリス）

世界遺産会議はユネスコ主催で年一回開催され、世界遺産の新規登録や登録物件の現状の評価などを行う。世界遺産の登録にあたっては、登録を求める地域の担当政府機関が候補地を推薦し、ユネスコ世界遺産委員会に暫定リストを提出する。委員会は国連機関に現地調査を依頼し、その結果を受けて登録推

In [14]:
import json
import gzip
with gzip.open("data/jawikinews-20170201.json.gz", "w") as fp:
    fp.write(json.dumps(dict(title=titles, text=contents), ensure_ascii=False).encode("utf-8"))